# 0.5_ss_chap6
6章の実装　


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
from src import utils
import itertools
import logging
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [4]:
log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
# logging.basicConfig(level=logging.INFO, format=log_fmt)
logging.basicConfig(level=logging.DEBUG, format=log_fmt)
logger = logging.getLogger()

# 中身

In [5]:
class SGD:
    ''' Simple stochasitc gradient descent
    '''
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]
    

### 関数を書いてみる

In [6]:
def bowl(x):
    ''' f(x,y) = 1/20 * x[0]^2 + x[1]^2
    ベクトルで入ってきた場合と、2次元（マトリックス）で入ってき場合に対応する
    
    Parameters
    ----------
    x: np.ndarray
        x values
    y: np.ndarray
        y values
    
    Returns
    -------
    f(x,y) = 1/20 * x^2 + y^2
    '''
    # takes numpy as input
    if type(x) != np.ndarray:
        raise ValueError("Only takes numpy.array as input")
        
    # 点
    if x.ndim == 1:
        return 1/20*x[0]**2 + x[1]**2
    
    # x should be 2 * N shape
    if x.shape[0] != 2:
        raise ValueError(f"x is {x.shape}, takes 2*N only")
    
    # ベクトル
    if x.ndim == 2:
        return 1/20*x[0] **2 + x[1]**2

    # それ以上　
    raise ValueError("Not impletented")
    


In [7]:
x = np.array([3, 5])

In [8]:
x.shape

(2,)

In [9]:
X = np.array([[3.,4.,3.],[5.,6.,7.]])

In [10]:
bowl(X) 

array([25.45, 36.8 , 49.45])

### 勾配を書いてみる

In [11]:
x0 = np.arange(-2, 2.5, .25)
x1 = np.arange(-2, 2.5, .25)
X, Y = np.meshgrid(x0, x1)
X = X.flatten()
Y = Y.flatten()

In [12]:
np.array([X, Y]).ndim

2

# あの numerical_gradient の中身はなんなんだ
- 一旦諦める

In [22]:
def _numerical_gradient_no_batch(f, x, debug=False):
        ''' Calcuate gradient at point x for f
        '''
        h = 1e-4
        grad = np.zeros_like(x)
        
        for idx in range(x.size):
                if debug and idx > 10:
                        break
                logger.debug(f"idx is {idx}")
                tmp_val = float(x[idx])
                logger.debug(f"tmp_val for x[idx] is {tmp_val}")
                # calculate f(x+h)
                x[idx] = tmp_val + h
                logger.debug(f"new x[idx] is {x[idx]}")
                fxh1 = f(x)
                logger.debug(f"f(x) is {fxh1}")
                
                # f(x-h)
                x[idx] = tmp_val - h
                logger.debug(f"new x[idx] is {x[idx]}")
                fxh2 = f(x)
                logger.debug(f"f(x) is {fxh2}")
                
                grad[idx] = (fxh1 - fxh2)/(2*h)
                logger.debug(f"grad[idx] is {grad[idx]}")
                x[idx] = tmp_val
        return grad

In [25]:
def numerical_gradient(f, X, debug=False):
        ''' Numerical gradient for 2d array
        Parameters
        ----------
        f: function
                function to calculate gradient
        X: 2d array
                grid to calculate gradient
        '''
        # return the original function if 1d
        logger.info(f"debug: {debug}")
        if X.ndim == 1:
                return _numerical_gradient_no_batch(f, X)
        
        grad = np.zeros_like(X)
        for idx, x in enumerate(X):
                logger.debug(f"idx is {idx}")
                grad[idx] = _numerical_gradient_no_batch(f, x, debug)
                
                # debug mode
                if debug and idx > 10:
                    return 
        return grad